In [1]:
# loading the required libraries
library(keras)
library(reticulate)
library(abind)
library(grid)

In [1]:
# loading the data and resizing them on the fly

train_path <- "data/flowers/"


image_width = 32
image_height = 32
target_image_size = c(image_width,image_height)


training_data <- flow_images_from_directory(directory = train_path,
                                        target_size = target_image_size,
                                        color_mode = "rgb",
                                        class_mode = NULL,
                                        batch_size = 2500)

training_data = as_iterator(training_data)
training_data = iter_next(training_data)
training_data <- training_data/255
dim(training_data)

In [6]:
# defining network parameters
latent_dim <- 32
height <- 32
width <- 32
channels <- 3

In [7]:
# creating the generator network
input_generator <- layer_input(shape = c(latent_dim))

output_generator <- input_generator %>% 
  
  # We transform the input data into a 16x16 128-channels feature map initially
  layer_dense(units = 128 * 16 * 16) %>%
  layer_activation_leaky_relu() %>% 
  layer_reshape(target_shape = c(16, 16, 128)) %>% 
  
  # Next ,we add a convolution layer
  layer_conv_2d(filters = 256, kernel_size = 5, 
                padding = "same") %>% 
  layer_activation_leaky_relu() %>% 
  
  # Now we upsample the data 
  layer_conv_2d_transpose(filters = 256, kernel_size = 4, 
                          strides = 2, padding = "same") %>% 
  layer_activation_leaky_relu() %>%
  
  # Now we add more convolutional layers to the network
  layer_conv_2d(filters = 256, kernel_size = 5, 
                padding = "same") %>% 
  layer_activation_leaky_relu() %>% 
  layer_conv_2d(filters = 256, kernel_size = 5, 
                padding = "same") %>% 
  layer_activation_leaky_relu() %>% 
  
  # Produce a 32x32 1-channel feature map
  layer_conv_2d(filters = channels, kernel_size = 7,
                activation = "tanh", padding = "same")

generator <- keras_model(input_generator, output_generator)

In [9]:
# summary of the generator network
summary(generator)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_1 (InputLayer)                (None, 32)                      0           
________________________________________________________________________________
dense (Dense)                       (None, 32768)                   1081344     
________________________________________________________________________________
leaky_re_lu (LeakyReLU)             (None, 32768)                   0           
________________________________________________________________________________
reshape (Reshape)                   (None, 16, 16, 128)             0           
________________________________________________________________________________
conv2d (Conv2D)                     (None, 16, 16, 256)             819456      
________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)   

In [10]:
# creating the discriminator network

input_discriminator <- layer_input(shape = c(height, width, channels))
output_discriminator <- input_discriminator %>% 
  layer_conv_2d(filters = 128, kernel_size = 3) %>% 
  layer_activation_leaky_relu() %>% 
#   layer_dropout(0.25) %>%
  layer_conv_2d(filters = 128, kernel_size = 4, strides = 2) %>% 
  layer_activation_leaky_relu() %>% 
#   layer_dropout(0.50) %>%
  layer_conv_2d(filters = 128, kernel_size = 4, strides = 2) %>% 
  layer_activation_leaky_relu() %>% 
  layer_conv_2d(filters = 128, kernel_size = 4, strides = 2) %>% 
  layer_activation_leaky_relu() %>% 
  layer_flatten() %>%
  # One dropout layer
  layer_dropout(rate = 0.3) %>%  
  # Classification layer
  layer_dense(units = 1, activation = "sigmoid")
discriminator <- keras_model(input_discriminator, output_discriminator)
summary(discriminator)

________________________________________________________________________________
Layer (type)                        Output Shape                    Param #     
input_2 (InputLayer)                (None, 32, 32, 3)               0           
________________________________________________________________________________
conv2d_4 (Conv2D)                   (None, 30, 30, 128)             3584        
________________________________________________________________________________
leaky_re_lu_5 (LeakyReLU)           (None, 30, 30, 128)             0           
________________________________________________________________________________
conv2d_5 (Conv2D)                   (None, 14, 14, 128)             262272      
________________________________________________________________________________
leaky_re_lu_6 (LeakyReLU)           (None, 14, 14, 128)             0           
________________________________________________________________________________
conv2d_6 (Conv2D)           

In [11]:
# compiling the discriminator network
discriminator %>% compile(
  optimizer = optimizer_rmsprop(lr = 0.0008,clipvalue = 1.0,decay = 1e-8),
  loss = "binary_crossentropy"
)

In [12]:
# freezing the weights of discriminator to make it non-trainable
freeze_weights(discriminator) 

In [13]:
# configuring the DCGAN network and compiling it
gan_input <- layer_input(shape = c(latent_dim),name = 'dc_gan_input')
gan_output <- discriminator(generator(gan_input))
gan <- keras_model(gan_input, gan_output)

gan %>% compile(
  optimizer = optimizer_rmsprop(lr = 0.0004,clipvalue = 1.0,decay = 1e-8), 
  loss = "binary_crossentropy"
)

In [ ]:
# summary of the GAN model
summary(gan)

In [14]:
# training our DCGAN network

# defining network parameters
iterations <- 2000
batch_size <- 40
dir.create("dcgan_images")
dir.create("dcgan_model")

Warning message in dir.create("dcgan_images"):
“'dcgan_images' already exists”Warning message in dir.create("dcgan_model"):
“'dcgan_model' already exists”

In [15]:
# Start the training
start_index <- 1

for (i in 1:iterations) {
  
  # Sample random points in the normally distributed latent space of dimension (batch_size * latent_dimension)
  random_latent_vectors <- matrix(rnorm(batch_size * latent_dim), 
                                  nrow = batch_size, ncol = latent_dim)
  
  # Use generator network to decode the above random points to fake images
  generated_images <- generator %>% predict(random_latent_vectors)
  
  # Combine the fake images with real images to build the training data for discriminator
  stop_index <- start_index + batch_size - 1 
  real_images <- training_data[start_index:stop_index,,,]
  rows <- nrow(real_images)
  combined_images <- array(0, dim = c(rows * 2, dim(real_images)[-1]))
  combined_images[1:rows,,,] <- generated_images
  combined_images[(rows+1):(rows*2),,,] <- real_images
 
  # Prvide appropriate labels for real and fake images
  labels <- rbind(matrix(1, nrow = batch_size, ncol = 1),
                  matrix(0, nrow = batch_size, ncol = 1))
  
    # Adds random noise to the labels to increase robustness of the discriminator
  labels <- labels + (0.5 * array(runif(prod(dim(labels))),
                                  dim = dim(labels)))
  
    # Train the discriminator using both real and fake images
  discriminator_loss <- discriminator %>% train_on_batch(combined_images, labels) 
  
    # Sample random points in the latent space
  random_latent_vectors <- matrix(rnorm(batch_size * latent_dim), 
                                  nrow = batch_size, ncol = latent_dim)
  
    # Assembles labels that say "all real images"
  misleading_targets <- array(0, dim = c(batch_size, 1))
  
    # Train the generator by using the gan model,note that the discriminator weights are frozen
  gan_model_loss <- gan %>% train_on_batch( 
    random_latent_vectors, 
    misleading_targets
  )  
  
  start_index <- start_index + batch_size
  if (start_index > (nrow(training_data) - batch_size))
    start_index <- 1
  
  # At few iterations save the model and save generated images

  if(i %in% c(5,10,15,20,40,100,200,500,800,1000,1500,2000)){

        # Save model
      save_model_hdf5(gan,paste0("dcgan_model/gan_model_",i,".h5"))

      # Save generated images
      generated_images <- generated_images *255
      generated_images  = array_reshape(generated_images ,dim = c(batch_size,32,32,3))
      generated_images  = (generated_images -min(generated_images ))/(max(generated_images )-min(generated_images ))
      grid = generated_images [1,,,]
      for(j in seq(2,5)){
          single = generated_images [j,,,]
          grid = abind(grid,single,along = 2)
      }
      png(file=paste0("dcgan_images/generated_flowers_",i,".png"),
      width=600, height=350)
      grid.raster(grid, interpolate=FALSE)
      dev.off()  
        
  }
}